In [39]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [40]:
curr_dir = '/home/jdtganding/Documents/data-engineering-zoomcamp/week_5_batch_processing'
df_green = spark.read.parquet(f'{curr_dir}/data/pq/green/*/*')
df_yellow = spark.read.parquet(f'{curr_dir}/data/pq/yellow/*/*')

We fix some of the data types and column names before merging both dataframes into one.

In [41]:
print(df_green.columns)
print('')
print(df_yellow.columns)

['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge', 'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge']

['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge']


In [42]:
#rename pickup and dropoff time
df_green = df_green.withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
                   .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

df_yellow = df_yellow.withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
                     .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [43]:
print(df_green.columns)

['VendorID', 'pickup_datetime', 'dropoff_datetime', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge', 'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge']


In [44]:
#find similar columns
print(set(df_green.columns) & set(df_yellow.columns))

{'RatecodeID', 'total_amount', 'store_and_fwd_flag', 'VendorID', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'DOLocationID', 'trip_distance', 'payment_type', 'pickup_datetime', 'PULocationID', 'fare_amount', 'mta_tax', 'extra', 'dropoff_datetime', 'passenger_count'}


In [45]:
#we preserve original column order
columns = []

for col in df_green.columns:
    if col in df_yellow.columns:
        columns.append(col)

print(columns)

['VendorID', 'pickup_datetime', 'dropoff_datetime', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'payment_type', 'congestion_surcharge']


In [46]:
from pyspark.sql import functions as F

#populate a column with the given service type
df_green = df_green.withColumn('service_type', F.lit('green'))
df_yellow = df_yellow.withColumn('service_type', F.lit('yellow'))

df_green.head(1)

[Row(VendorID=2, pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), dropoff_datetime=datetime.datetime(2020, 1, 23, 13, 38, 16), store_and_fwd_flag='N', RatecodeID=1, PULocationID=74, DOLocationID=130, passenger_count=1, trip_distance=12.77, fare_amount=36.0, extra=0.0, mta_tax=0.5, tip_amount=2.05, tolls_amount=6.12, ehail_fee=None, improvement_surcharge=0.3, total_amount=44.97, payment_type=1, trip_type=1, congestion_surcharge=0.0, service_type='green')]

In [52]:
#select only all needed columns
columns.append('service_type')
df_green_sel = df_green.select(columns)
df_yellow_sel = df_yellow.select(columns)

#merge both dataframes
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [53]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [55]:
#we can create a temp table using our dataframe
df_trips_data.registerTempTable('trips_data')

In [59]:
spark.sql("""
SELECT 
    service_type,
    count(*) as count
FROM trips_data
GROUP BY service_type
""").show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [61]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [62]:
#we can save the results as a parquet file and use coalesce to reduce the partition
df_result.coalesce(1).write.parquet('data/report/revenue', mode='overwrite')